In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 25.5 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 14.6 MB 20.8 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')


In [2]:
import torch

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DEVICE

device(type='cuda')

# Data Load

In [3]:
def file_load(path):
    data_path = []
    f = open(path, 'r')
    while True:
        line = f.readline()
        if not line:
            break
        data_path.append(line[:-1])
    f.close()
    return data_path

In [4]:
import torch.utils.data as data

class CustomDataset(data.Dataset):
    def __init__(self, path_data):
        super(CustomDataset, self).__init__()
        self.src = file_load('{0}.en'.format(path_data))
        self.trg = file_load('{0}.de'.format(path_data))

    def __getitem__(self, index):
        return self.src[index], self.trg[index]
    
    def __len__(self):
        return len(self.src)

In [5]:
import torch
import torch.utils.data as data
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab
from torch.nn.utils.rnn import pad_sequence


en_tokenizer = get_tokenizer('spacy', language='en_core_web_sm')
de_tokenizer = get_tokenizer('spacy', language='de_core_news_sm')

dpath = 'drive/MyDrive/IIPL/p-transformer/europarl-v7.de-en'
cdata = CustomDataset(dpath)

In [6]:
from collections import Counter
from torchtext.vocab import Vocab

def build_vocab(data, tokenizer):
    counter = Counter()
    for line in data:
        counter.update(tokenizer(line))
    return Vocab(counter)

In [7]:
en_vocab = build_vocab(cdata.src, en_tokenizer)
de_vocab = build_vocab(cdata.trg, de_tokenizer)

In [8]:
def _collate_fn(batch):
    src_list, trg_list = [], []

    for text in batch:
        processed_src = torch.tensor([en_vocab[token] for token in text[0]], dtype=torch.int64)
        processed_trg = torch.tensor([de_vocab[token] for token in text[1]], dtype=torch.int64)
        src_list.append(processed_src)
        trg_list.append(processed_trg)
    
    src_list = pad_sequence(src_list, batch_first=True, padding_value=0)
    trg_list = pad_sequence(trg_list, batch_first=True, padding_value=0)

    return src_list, trg_list


class CustomDataLoader(data.DataLoader):
    def __init__(self, *args, **kwargs):
        super(CustomDataLoader, self).__init__(*args, **kwargs)
        self.collate_fn = _collate_fn

In [9]:
dataloader = CustomDataLoader(cdata, batch_size=64, shuffle=False)

In [10]:
for i, data in enumerate(dataloader):
    print(data[0])  # src 문장 64개
    print(data[0].size())
    print(data[1])  # trg 문장 64개
    print(data[1].size())
    break

tensor([[    46,   1435,  17954,  ...,      0,      0,      0],
        [527803,      0,    136,  ...,      0,      0,      0],
        [ 17598,     36,    239,  ...,      0,      0,      0],
        ...,
        [527803,    239,      0,  ...,      0,      0,      0],
        [ 17598,  17954,      0,  ...,      0,      0,      0],
        [   172,     28,   1435,  ...,      0,      0,      0]])
torch.Size([64, 402])
tensor([[ 10, 566, 223,  ...,   0,   0,   0],
        [967, 179,  94,  ...,   0,   0,   0],
        [ 10, 566, 223,  ...,   0,   0,   0],
        ...,
        [241, 566, 223,  ...,   0,   0,   0],
        [ 10, 566, 223,  ...,   0,   0,   0],
        [155, 102,   0,  ...,   0,   0,   0]])
torch.Size([64, 410])


In [12]:
l_src, l_trg = [], []
for i, data in enumerate(dataloader):
    l_src.append(data[0].size()[1])
    l_trg.append(data[1].size()[1])

In [13]:
print(max(l_src), max(l_trg))   # src, trg 최대 토큰 길이

3950 3631


In [14]:
BATCH_SIZE = 64
TEST_SPLIT = 0.2
VAL_SPLIT = 0.2
SHUFFLE = True
RANDOM_SEED = 42

In [15]:
import numpy as np
from torch.utils.data.sampler import SubsetRandomSampler

data_size = len(cdata)  #100
indices = list(range(data_size))
test_split = int(np.floor(TEST_SPLIT * data_size))  #20
val_split = int(np.floor(VAL_SPLIT * (data_size-test_split))) + test_split   #16+20
if SHUFFLE:
    np.random.seed(RANDOM_SEED)
    np.random.shuffle(indices)

# [16+20:], [20:16+20], [:20] 
train_idx, val_idx, test_idx = indices[val_split:], indices[test_split:val_split], indices[:test_split]

train_sampler = SubsetRandomSampler(train_idx)
val_sampler = SubsetRandomSampler(val_idx)
test_sampler = SubsetRandomSampler(test_idx)

train_iter = CustomDataLoader(cdata, batch_size=BATCH_SIZE, sampler=train_sampler, shuffle=False)
val_iter = CustomDataLoader(cdata, batch_size=BATCH_SIZE, sampler=val_sampler, shuffle=False)
test_iter = CustomDataLoader(cdata, batch_size=BATCH_SIZE, sampler=test_sampler, shuffle=False)

In [16]:
print(len(train_iter), len(val_iter), len(test_iter))

19203 4801 6001


# Utils

In [17]:
import copy
import torch.nn as nn

def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

# Model Architecture

## embedding

In [18]:
import math
import torch.nn as nn

class Embeddings(nn.Module):
    def __init__(self, d_model, vocab):
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        self.d_model = d_model

    def forward(self, x):
        return self.lut(x) * math.sqrt(self.d_model)

In [19]:
import torch
import torch.nn as nn
from torch.autograd import Variable

class PositionalEncoding(nn.Module):
    "Implement the PE function."
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        
        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)], requires_grad=False)
        return self.dropout(x)

## genreator

In [20]:
import torch.nn as nn
import torch.nn.functional as F

class Generator(nn.Module):
    "Define standard linear + softmax generation step."
    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        return F.log_softmax(self.proj(x), dim=-1)

## layer

In [21]:
import torch
import torch.nn as nn

class MultiHeadedAttention(nn.Module):
    def __init__(self, n_heads, hid_dim, device, dropout=0.1):
        "Take in model size and number of heads."
        super(MultiHeadedAttention, self).__init__()
        assert hid_dim % n_heads == 0
        # We assume d_v always equals head_dim
        self.head_dim = hid_dim // n_heads
        self.n_heads = n_heads
        self.hid_dim = hid_dim

        self.fc_q = nn.Linear(hid_dim, hid_dim)
        self.fc_k = nn.Linear(hid_dim, hid_dim)
        self.fc_v = nn.Linear(hid_dim, hid_dim)

        self.fc_o = nn.Linear(hid_dim, hid_dim)

        self.dropout = nn.Dropout(p=dropout)

        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)
        
    def forward(self, query, key, value, mask=None):

        batch_size = query.shape[0]

        Q = self.fc_q(query)
        K = self.fc_k(key)
        V = self.fc_v(value)

        Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0,2,1,3)
        K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0,2,1,3)
        V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0,2,1,3)

        energy = torch.matmul(Q, K.permute(0,1,3,2)) / self.scale

        if mask is not None:
            energy = energy.masked_fill(mask==0, -1e10)
        attention = torch.softmax(energy, dim=-1)

        x = torch.matmul(self.dropout(attention),V)
        x = x.permute(0,2,1,3).contiguous()
        x = x.view(batch_size, -1, self.hid_dim)
        x = self.fc_o(x)

        return x, attention

In [22]:
import torch.nn as nn

class PositionwiseFeedForward(nn.Module):
    "Implements FFN equation."
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))

In [23]:
import torch
import torch.nn as nn

class LayerNorm(nn.Module):
    "Construct a layernorm module."
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

In [24]:
import torch.nn as nn

class SublayerConnection(nn.Module):
    """
    A residual connection followed by a layer norm.
    Note for code simplicity the norm is first as opposed to last.
    """
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."
        drop = sublayer(self.norm(x))
        return x + self.dropout(drop[0])

## model

In [25]:
class EncoderLayer(nn.Module):
    "Encoder is made up of self-attn and feed forward (defined below)"
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask):
        "Follow Figure 1 (left) for connections."
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)

In [26]:
class Encoder(nn.Module):
    "Core encoder is a stack of N layers"
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, mask):
        "Pass the input (and mask) through each layer in turn."
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

In [27]:
class DecoderLayer(nn.Module):
    "Decoder is made of self-attn, src-attn, and feed forward (defined below)"
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 3)
 
    def forward(self, x, memory, src_mask, tgt_mask):
        "Follow Figure 1 (right) for connections."
        m = memory
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))
        return self.sublayer[2](x, self.feed_forward)

In [28]:
class Decoder(nn.Module):
    "Generic N layer decoder with masking."
    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)
        
    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)

In [29]:
class EncoderDecoder(nn.Module):
    """
    A standard Encoder-Decoder architecture. Base for this and many 
    other models.
    """
    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator
        
    def forward(self, src, tgt, src_mask, tgt_mask):
        "Take in and process masked src and target sequences."
        return self.decode(self.encode(src, src_mask), src_mask,
                            tgt, tgt_mask)
    
    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask)
    
    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

In [30]:
def make_model(src_vocab, tgt_vocab, N=6, 
               d_model=512, d_ff=2048, h=8, dropout=0.1, device='cpu'):
    "Helper: Construct a model from hyperparameters."
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model, device)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    model = EncoderDecoder(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), 
                             c(ff), dropout), N),
        nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
        nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
        Generator(d_model, tgt_vocab))
    
    # This was important from their code. 
    # Initialize parameters with Glorot / fan_avg.
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform(p)
    return model

In [31]:
def subsequent_mask(size):
    "Mask out subsequent positions."
    attn_shape = (1, size, size)
    subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')
    return torch.from_numpy(subsequent_mask) == 0


class Batch:
    "Object for holding a batch of data with mask during training."
    def __init__(self, src, trg=None, pad=0):
        self.src = src
        self.src_mask = (src != pad).unsqueeze(-2)
        if trg is not None:
            self.trg = trg
            self.trg_mask = self.make_std_mask(self.trg, pad)
            self.ntokens = (self.trg != pad).data.sum()
    
    @staticmethod
    def make_std_mask(tgt, pad):
        "Create a mask to hide padding and future words."
        tgt_mask = (tgt != pad).unsqueeze(-2)
        tgt_mask = tgt_mask & Variable(subsequent_mask(tgt.size(-1)).type_as(tgt_mask.data))
        return tgt_mask

In [32]:
import time

def run_epoch(data_iter, model, loss_compute):
    "Standard Training and Logging Function"
    start = time.time()
    total_tokens = 0
    total_loss = 0
    tokens = 0
    for i, data in enumerate(data_iter):
        batch = Batch(data[0], data[1])
        out = model.forward(batch.src, batch.trg, batch.src_mask, batch.trg_mask)
        loss = loss_compute(out, batch.trg_y, batch.ntokens)
        total_loss += loss
        total_tokens += batch.ntokens
        tokens += batch.ntokens
        if i % 50 == 1:
            elapsed = time.time() - start
            print("Epoch Step: %d Loss: %f Tokens per Sec: %f" %
                    (i, loss / batch.ntokens, tokens / elapsed))
            start = time.time()
            tokens = 0
    return total_loss / total_tokens

In [33]:
from torchtext.data.metrics import bleu_score

model = make_model(len(en_vocab), len(de_vocab), N=2)
run_epoch(val_iter, model, bleu_score)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.


IndexError: ignored